# Intervention keys

In [1]:
import pandas as pd
import random
import numpy as np

In [75]:
# General config & paths

from pathlib import Path

# Project root = folder where this notebook lives
PROJECT_ROOT = Path().resolve()

# Data folders (relative paths)
DATA_DIR = PROJECT_ROOT / "data"
QUALTRICS_DIR = DATA_DIR / "qualtrics"
WAVE2_DIR = QUALTRICS_DIR / "wave2"
KEYS_DIR = DATA_DIR / "keys"
MESSAGES_DIR = DATA_DIR / "messages"

In [31]:
parents_w2 = pd.read_csv(WAVE2_DIR / "Wave 2 - parents.csv")
date_change_w2 = pd.read_csv(WAVE2_DIR /"Wave 2 - date change.csv")
responses_w2 = pd.read_csv(WAVE2_DIR /"Wave 2 - responses.csv")

parents_w2["StartDate"] = pd.to_datetime(parents_w2["StartDate"], errors='coerce')
date_change_w2["StartDate"] = pd.to_datetime(date_change_w2["StartDate"], errors='coerce')
#filter_date = pd.to_datetime("2024-10-24 00:00:00")
#parents_w2 = parents_w2[parents_w2["StartDate"] > filter_date]

# Sort by 'Start Date' in descending order so the most recent dates come first
date_change_w2 = date_change_w2.sort_values(by='StartDate', ascending=False)
responses_w2 = responses_w2[2:]


C:\Users\77197jsc\AppData\Local\Temp\ipykernel_19004\1496548962.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parents_w2["StartDate"] = pd.to_datetime(parents_w2["StartDate"], errors='coerce')
C:\Users\77197jsc\AppData\Local\Temp\ipykernel_19004\1496548962.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_change_w2["StartDate"] = pd.to_datetime(date_change_w2["StartDate"], errors='coerce')


In [32]:
parents_w2["Game_preview_exp"] = parents_w2["Game_preview_exp"].fillna(parents_w2['Game_preview_ctr'])
parents_w2['Mail_child_exp'] = parents_w2['Mail_child_exp'].fillna(parents_w2['Mail_child_ctr'])
parents_w2 = parents_w2.reset_index(drop=True)
print(parents_w2["Game_preview_exp"].isna().sum())
print(len(responses_w2))

178
54


## Qualtrics data

In [33]:
responses_w2.columns

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Child_ID', 'Mail_child',
       'Imposter', 'Game_pleasure', 'Game_relatedness', 'Game_capability',
       'Sharing_behavior_gam', 'Game_rating', 'Game_feedback', 'Sex_1',
       'Sex_2', 'Sex_3', 'Sex_4', 'Gender', 'Gender_11_TEXT', 'App_prep',
       'Random ID'],
      dtype='object')

In [34]:
parents_w2["Q3"].value_counts()

Q3
1                               50
2                                4
Informed consent                 1
{"ImportId":"QID1216613996"}     1
Name: count, dtype: int64

In [35]:
keys_filtered = parents_w2[(parents_w2['Progress'] == "100") & ((parents_w2['Informed_consent_exp'] == '1') | (parents_w2['Q3'] == '1'))]

len(keys_filtered)

172

In [36]:
keys_w2 = keys_filtered["id"].reset_index()

#keys_w2.to_csv(DATA_DIR / "keys" / "keys_w2.csv")

In [37]:
# Add column for being in control group (0 = no, 1 = yes)
keys_filtered['control'] = keys_filtered['Q3'].apply(lambda x: 1 if x == "1" else 0)

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_19004\4146169195.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keys_filtered['control'] = keys_filtered['Q3'].apply(lambda x: 1 if x == "1" else 0)


In [38]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

keys_filtered['Random ID'] = keys_filtered['Random ID'].astype(float)
responses_w2['Child_ID']= responses_w2['Child_ID'].astype(float)

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_19004\536526801.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keys_filtered['Random ID'] = keys_filtered['Random ID'].astype(float)


In [39]:
# Matches in keys_filtered → responses_w2
match_by_mail_keys = keys_filtered["Mail_child_exp"].isin(responses_w2["Mail_child"])
match_by_id_keys   = keys_filtered["Random ID"].isin(responses_w2["Child_ID"])

print("Keys matched by mail:", match_by_mail_keys.sum())
print("Keys matched by ID:  ", match_by_id_keys.sum())

filtered_keys = keys_filtered[match_by_mail_keys | match_by_id_keys]
print("Total matched key rows:", len(filtered_keys))


# Matches in responses_w2 → keys_filtered (to see what can't be linked)
match_by_mail_resp = responses_w2["Mail_child"].isin(keys_filtered["Mail_child_exp"])
match_by_id_resp   = responses_w2["Child_ID"].isin(keys_filtered["Random ID"])

neither_mask = ~(match_by_mail_resp | match_by_id_resp)
neither = responses_w2[neither_mask]

print("Unmatched response rows:", len(neither))


Keys matched by mail: 34
Keys matched by ID:   50
Total matched key rows: 52
Unmatched response rows: 3


In [40]:
# Assign a random number (arms 1-4) for people that are not in the control group
filtered_keys['condition'] = filtered_keys['control'].apply(lambda x: random.randint(2, 4) if x == 0 else 1)

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_19004\3742310706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_keys['condition'] = filtered_keys['control'].apply(lambda x: random.randint(2, 4) if x == 0 else 1)


In [41]:
# Value 4 needs to be split up into two group 4 and 5 which assigns the order of tailored/non-tailored intervention

def random_replace(value):
    if value == 4:
        return np.random.choice([4, 5])
    return value

# Apply the function to the column
filtered_keys['condition'] = filtered_keys['condition'].apply(random_replace)

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_19004\2709943707.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_keys['condition'] = filtered_keys['condition'].apply(random_replace)


In [42]:
filtered_keys = filtered_keys[['Mail_child_exp', 'Mail_child_ctr', 'id', 'Random ID', 'condition']]

filtered_keys = filtered_keys.rename(columns={"id": "Norstat ID", "Random ID": "Movez ID"})

#Collapse mails into a single column
filtered_keys['Mail_child_exp'] = filtered_keys['Mail_child_exp'].fillna(filtered_keys['Mail_child_ctr'])

In [43]:
filtered_keys_w2 = filtered_keys.drop(columns = "Mail_child_ctr")

filtered_keys_w2 = filtered_keys_w2.reset_index()

In [44]:
filtered_keys_w2

,index,Mail_child_exp,Norstat ID,Movez ID,condition
0,4,irishvader@outlook.com,yNedoa0VX45AMEjw,97077.0,3
1,21,chisommaria5@gmail.com,3YVN2ManXjp2bRn5,17970.0,1
2,22,Gizmo910@icloud.com,NaBzAB8D4Qw2YyVM,28371.0,5
3,58,gaagoobilo@gmail.com,Vw142r07XqPAaNPX,64938.0,3
4,59,likohgre@gmail.com,BdVOAGVQVXE968JL,18557.0,2
5,60,romeobenson23@gmail.com,5nJ4ApbP4Ne2G3kx,43876.0,3
6,97,Jk0284@outlook.com,NaBzAB8BqyO2YyVM,77982.0,1
7,98,jessprueitt37@gmail.com,D5je9bGk6zLARMYl,57519.0,3
8,102,gbucci72@gmail.com,jnWOo0XGGJXoXzex,11478.0,2
9,106,Jlcawthray@hotmail.com,ybDaoyd0gzNAQdvY,64720.0,3


In [20]:
filtered_keys["condition"].value_counts()

2    20
4    14
3    10
1     8
Name: condition, dtype: int64

In [45]:
#filtered_keys_w2.to_csv(DATA_DIR / "keys" / "qualtrics_keys_w2.csv", index=False)

In [46]:
#filtered_keys_w2.to_csv(DATA_DIR / "keys" / "qualtrics_keys_w12_new.csv", index=False)

### Avicenna keys

In [49]:
qualtrics_keys_w2 = pd.read_csv(KEYS_DIR / "wave2" / "qualtrics_keys_w12_new.csv")

# Loading the Avicenna participation file
avicenna = pd.read_csv(KEYS_DIR / "wave2" /'study-3843-export-32-participation-2024-11-18-09-11-48.csv')

In [50]:
avicenna.head()

,ID,Participant Joined Date,Start Time,End Time,Label,Status,Last Recorded Data Time,In Operation,Participant Type,First Name,Last Name,Email Address,Devices,Session Stats
0,48610,NaN,2024-09-06 10:16:44.558000+00:00,2024-10-08 11:29:29.239000+00:00,Ying,3,2024-09-17 17:10:00+02:00,0,Main,Ying,NaN,ying_chuck@hotmail.com,"[{""id"":""c7ad9571657e1bac"",""manufacturer"":""Xiao...","{""unknown"":0,""completed"":2,""canceled"":0,""expir..."
1,96792,NaN,2024-08-29 21:07:20.062000+00:00,2970-06-01 00:00:00+00:00,Sara,1,2024-10-10 03:52:58+03:30,0,Main,S,F,sfarnya+4@gmail.com,"[{""id"":""b03c915ba8b154c1"",""manufacturer"":""sams...","{""unknown"":5,""completed"":6,""canceled"":0,""expir..."
2,97152,NaN,2024-06-11 13:47:37.984000+00:00,2970-06-01 00:00:00+00:00,David,1,2024-09-17 21:55:00+02:00,0,Main,David,Blok,d.j.blok@essb.eur.nl,"[{""id"":""3C22644134484E6F88459A81394BF940"",""man...","{""unknown"":0,""completed"":15,""canceled"":2,""expi..."
3,101115,NaN,2024-09-04 15:20:11.859000+00:00,2970-06-01 00:00:00+00:00,Jonas,1,2024-10-04 18:45:56+02:00,0,Main,BU,"[UNIQUE ID, See Mail]",bla@böa.com,"[{""id"":""D0D97059E4674A3288F616A2AB9CA531"",""man...","{""unknown"":0,""completed"":9,""canceled"":6,""expir..."
4,101307,NaN,2024-09-03 10:24:35.813000+00:00,2024-10-08 11:29:29.399000+00:00,Kim,3,2024-09-12 13:30:00+02:00,0,Main,Kim,Doornhein,kimdoornhein99@gmail.com,"[{""id"":""E3262DCCC664470A8484DD98EDF7824B"",""man...","{""unknown"":0,""completed"":3,""canceled"":0,""expir..."


In [51]:
# Selecting relevant columns
avicenna = avicenna[["ID", "Last Name", "Email Address"]]

avicenna = avicenna.rename(columns={"Last Name": "Movez ID", "Email Address": "Mail_child_exp"})


# Only selecting particiaptions that have a valid 5-digit ID
avicenna['Movez ID'] = pd.to_numeric(avicenna['Movez ID'], errors='coerce').fillna(0)

avicenna = avicenna[8:]

print(len(avicenna))

50


In [52]:
# First merge on "Movez Code"
merged_movez_code = avicenna.merge(qualtrics_keys_w2, on="Movez ID", how="left", suffixes=('', '_movez'))

# Second merge on "Mail_child_exp" where there are still NaN values from the first merge
merged_mail_child_exp = merged_movez_code.merge(
    qualtrics_keys_w2, 
    on="Mail_child_exp", 
    how="left", 
    suffixes=('', '_mail')
)

# Use columns from the first merge if present, otherwise from the second merge
for col in qualtrics_keys_w2.columns:
    if col != "Movez Code" and col != "Mail_child_exp":
        merged_mail_child_exp[col] = merged_mail_child_exp[col].combine_first(merged_mail_child_exp[f"{col}_mail"])

# Drop the duplicate columns created by the second merge
merged_mail_child_exp = merged_mail_child_exp.drop(columns=[f"{col}_mail" for col in qualtrics_keys_w2.columns if col != "Movez Code" and col != "Mail_child_exp"])

print(len(merged_mail_child_exp))

52


In [53]:
# Manual corrections for Norstat ID
norstat_fixes = {
    44: "ybDaoyd0gzNAQdvY",
    49: "Vj6N985vvQ1A8drl",
}

# Manual corrections for Movez ID
movez_fixes = {
    45: 98620.0,
    34: 34338.0,
    30: 19593.0,
}

for idx, val in norstat_fixes.items():
    merged_mail_child_exp.at[idx, "Norstat ID"] = val

for idx, val in movez_fixes.items():
    merged_mail_child_exp.at[idx, "Movez ID"] = val


In [54]:
merged_mail_child_exp_w2 = merged_mail_child_exp[7:]

In [55]:
merged_mail_child_exp_w2["condition"].value_counts()

condition
2.0    16
3.0    10
5.0     7
1.0     7
4.0     4
Name: count, dtype: int64

In [56]:
merged_mail_child_exp_w2 = merged_mail_child_exp_w2.drop_duplicates()

keys_w2 = merged_mail_child_exp_w2

In [57]:
# Put myself into the control condition
keys_w2.at[51, "condition"] = 1.0
keys_w2.at[51, "Movez ID"] = 11111

In [58]:
keys_w2["ID"] = keys_w2["ID"].astype(float)

In [59]:
keys_w2

,ID,Movez ID,Mail_child_exp,index,Mail_child_exp_movez,Norstat ID,condition
7,106521.0,12964.0,nabz80@hotmail.com,283.0,Malaika2007x@hotmail.com,LDen9K41b1QAElVw,3.0
8,106522.0,70186.0,joe.mid2011@outlook.com,198.0,joemid2011@gmail.com,YXrqo4l375NoMZdm,5.0
9,106523.0,45582.0,tolaaudio@gmail.com,152.0,Tolaaudio@gmail.com,81K7AmYRz66AZmyO,3.0
10,106524.0,45131.0,ruthmills207@yahoo.com,261.0,Ruthmills207@yahoo.com,RWLV2ZQqQjzo5vN3,4.0
11,106526.0,70634.0,peereez2099@gmail.com,246.0,peereez2099@gmail.com,zExm2JmdXQ1o6PyK,5.0
12,106527.0,16575.0,walshalex001@gmail.com,247.0,walshalex001@gmail.com,Elb8AvB1Dv5ApLX5,5.0
13,106531.0,63044.0,beexbeey@gmail.com,144.0,adeotiobanijesu@gmail.com,QGYe2jXzkV5AgrXp,1.0
14,106532.0,63722.0,christopher_fullard@yahoo.co.uk,219.0,christopher_fullard@yahoo.co.uk,qRZK2gj6ebyobdka,1.0
15,106533.0,0.0,sxhamilt@gmail.com,129.0,NaN,nKY7owOmG8noG3v4,2.0
16,106555.0,42637.0,samfisherhat@gmail.com,250.0,samfisherhat@gmail.com,nKY7oww4YNdoG3v4,2.0


In [60]:
#Create Mail List

keys = list(keys_w2["Mail_child_exp"])

"; ".join(keys)

'nabz80@hotmail.com; joe.mid2011@outlook.com; tolaaudio@gmail.com; ruthmills207@yahoo.com; peereez2099@gmail.com; walshalex001@gmail.com; beexbeey@gmail.com; christopher_fullard@yahoo.co.uk; sxhamilt@gmail.com; samfisherhat@gmail.com; sxhamill@gmail.com; elvisboy87@gmail.com; maryjonathan716@gmail.com; tommyhills9900@gmail.com; millieobura6@gmail.com; historianden@gmail.com; magdisabry906@gmail.com; richardfadoju89@gmail.com; bludrilko@gmail.com; okewunmifemi2022@gmail.com; caroline.mckenzie@hotmail.co.uk; gbucci72@gmail.com; btechmmust@gmail.com; jaspercrane12@gmail.com; simonhastings27@gmail.com; loveleadproject@gmail.com; faithsimiyu42@gmail.com; bethsuccessful@gmail.com; tahmima@hotmail.com; mikkie2014@outlook.com; ethanmc2012@hotmail.com; eugenewonguk@gmail.com; jessprueitt37@gmail.com; williamsjames3747@gmail.com; felicityadiza31@gmail.com; jk0284@outlook.com; jlcawthray@hotmail.com; ladyshumz1@hotmail.com; immart01@aol.com; hyacinthyoung25@gmail.com; abdullservant@gmail.com; dia

In [61]:
#keys_w22 = keys_w2[["ID", "condition"]]
#keys_w2.to_csv(KEYS_DIR / "keys_w2.csv")

In [62]:
# Let's filter to only keep Avicenna users that signed up in the present wave

# Create boolean masks for "Movez ID" and "Email Address" matches
id_match = merged_mail_child_exp["Movez ID"].isin(qualtrics_keys_w2["Movez ID"])
mail_match = merged_mail_child_exp["Mail_child_exp"].isin(qualtrics_keys_w2["Mail_child_exp"])

# Filter dataframe1 to keep rows where either condition is true
avicenna_filtered = merged_mail_child_exp[id_match | mail_match]

avicenna_filtered = avicenna_filtered.reset_index()

In [63]:
avicenna_filtered

,level_0,ID,Movez ID,Mail_child_exp,index,Mail_child_exp_movez,Norstat ID,condition
0,1,104582,43876.0,romeobenson23@gmail.com,60.0,romeobenson23@gmail.com,5nJ4ApbP4Ne2G3kx,4.0
1,2,104584,64938.0,gaagoobilo@gmail.com,58.0,gaagoobilo@gmail.com,Vw142r07XqPAaNPX,3.0
2,3,104605,97077.0,irishvader@outlook.com,4.0,irishvader@outlook.com,yNedoa0VX45AMEjw,2.0
3,4,104608,18557.0,likohgre@gmail.com,59.0,likohgre@gmail.com,BdVOAGVQVXE968JL,3.0
4,5,104702,28371.0,gizmo910@icloud.com,22.0,Gizmo910@icloud.com,NaBzAB8D4Qw2YyVM,5.0
5,6,104711,17970.0,chisommaria5@gmail.com,21.0,chisommaria5@gmail.com,3YVN2ManXjp2bRn5,1.0
6,7,106521,12964.0,nabz80@hotmail.com,283.0,Malaika2007x@hotmail.com,LDen9K41b1QAElVw,3.0
7,8,106522,70186.0,joe.mid2011@outlook.com,198.0,joemid2011@gmail.com,YXrqo4l375NoMZdm,5.0
8,9,106523,45582.0,tolaaudio@gmail.com,152.0,Tolaaudio@gmail.com,81K7AmYRz66AZmyO,3.0
9,10,106524,45131.0,ruthmills207@yahoo.com,261.0,Ruthmills207@yahoo.com,RWLV2ZQqQjzo5vN3,4.0


## Reponse rate

In [65]:
response_w2 = pd.read_csv(KEYS_DIR / 'wave2' / 'study-3843-activity-sessions-2024-11-26-14-27-24.csv')
response_w2 = response_w2[["Activity ID", "User ID", "Scheduled Time", "Status"]]
#keys_w2 = pd.read_csv("keys_w2.csv")

response_w2["Scheduled Time"] = pd.to_datetime(response_w2["Scheduled Time"], errors='coerce')
# Define the date range
start_date = '2024-11-01'
end_date = '2024-11-30'

response_w2 = response_w2[(response_w2['Scheduled Time'] >= start_date) & (response_w2['Scheduled Time'] <= end_date)]

In [66]:
response_w2 = response_w2.merge(keys_w2, left_on = "User ID", right_on="ID", how= "left")

response_w2.head()

,Activity ID,User ID,Scheduled Time,Status,ID,Movez ID,Mail_child_exp,index,Mail_child_exp_movez,Norstat ID,condition
0,22395,106521,2024-11-11 06:30:00+00:00,Expired,106521.0,12964.0,nabz80@hotmail.com,283.0,Malaika2007x@hotmail.com,LDen9K41b1QAElVw,3.0
1,22395,106522,2024-11-11 06:30:00+00:00,Expired,106522.0,70186.0,joe.mid2011@outlook.com,198.0,joemid2011@gmail.com,YXrqo4l375NoMZdm,5.0
2,22395,106523,2024-11-11 06:30:00+00:00,Completed,106523.0,45582.0,tolaaudio@gmail.com,152.0,Tolaaudio@gmail.com,81K7AmYRz66AZmyO,3.0
3,22395,106524,2024-11-11 06:30:00+00:00,Completed,106524.0,45131.0,ruthmills207@yahoo.com,261.0,Ruthmills207@yahoo.com,RWLV2ZQqQjzo5vN3,4.0
4,22395,106526,2024-11-11 05:30:00+00:00,Completed,106526.0,70634.0,peereez2099@gmail.com,246.0,peereez2099@gmail.com,zExm2JmdXQ1o6PyK,5.0


In [67]:
#Lets fill in numerical codes

response_w2["Status"] = response_w2['Status'].replace({
    'Completed': 1,
    'Unknown': 99,
    'Expired': 0,
    'Canceled': 99,
    'Blocked': 99,
     'NaN': 99})

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_19004\3617494394.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  response_w2["Status"] = response_w2['Status'].replace({


In [68]:
# Pivot responses: each Activity ID becomes a separate column with Status values
response_w2_wide = (
    response_w2
    .pivot(index="User ID", columns="Activity ID", values="Status")
    .reset_index()
)

# Merge in Norstat ID + condition from keys
response_w2_wide = response_w2_wide.merge(
    keys_w2[["ID", "Norstat ID", "condition"]],
    left_on="User ID",
    right_on="ID",
    how="left"
)

#drop duplicate ID column if you don't need both
response_w2_wide = response_w2_wide.drop(columns=["ID"])


In [76]:
response_w2_wide.head()

,condition,User ID,22395,22404,22405,22406,22407,22408,22409,22410,22411,22412,Baseline,Weekend 1,Weekend 2,22606,22607,22608,22609,22610,22611,22612,22613,22614,22716,Norstat ID,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10
0,3.0,106521,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,LDen9K41b1QAElVw,0,0,1,1,1,1,0,0,0,0
1,5.0,106522,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,YXrqo4l375NoMZdm,0,0,0,0,0,1,1,1,1,1
2,3.0,106523,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,81K7AmYRz66AZmyO,1,1,1,1,1,0,0,1,1,1
3,4.0,106524,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,RWLV2ZQqQjzo5vN3,1,1,1,1,1,1,1,1,1,1
4,5.0,106526,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,zExm2JmdXQ1o6PyK,1,1,1,1,1,1,1,1,1,1


In [70]:
pairs = [
    ([22395, 22716], 'Day 1'),
    ([22606, 22404], 'Day 2'),
    ([22607, 22405], 'Day 3'),
    ([22608, 22406], 'Day 4'),
    ([22609, 22407], 'Day 5'),
    ([22610, 22408], 'Day 6'),
    ([22611, 22409], 'Day 7'),
    ([22612, 22410], 'Day 8'),
    ([22613, 22411], 'Day 9'),
    ([22614, 22412], 'Day 10',),
]

#for columns, day_name in pairs:
    #pilot_resp_wide[day_name] = ((pilot_resp_wide[columns[0]] == 1) & (pilot_resp_wide[columns[1]] == 1)).astype(int)

for columns, day_name in pairs:
    response_w2_wide[day_name] = (
        ((response_w2_wide['condition'] == 1) & (response_w2_wide[columns[0]] == 1)) |
        ((response_w2_wide['condition'] != 1) & (response_w2_wide[columns[0]] == 1) & (response_w2_wide[columns[1]] == 1))
    ).astype(int)

response_w2_wide = response_w2_wide.rename(columns={22414: "Baseline", 22415: "Weekend 1", 22419: "Weekend 2"})

In [77]:
column_to_front = "condition"
cols = [column_to_front] + [col for col in response_w2_wide.columns if col != column_to_front]
response_w2_wide = response_w2_wide[cols]

response_w2_wide.head()

,condition,User ID,22395,22404,22405,22406,22407,22408,22409,22410,22411,22412,Baseline,Weekend 1,Weekend 2,22606,22607,22608,22609,22610,22611,22612,22613,22614,22716,Norstat ID,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10
0,3.0,106521,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,LDen9K41b1QAElVw,0,0,1,1,1,1,0,0,0,0
1,5.0,106522,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,YXrqo4l375NoMZdm,0,0,0,0,0,1,1,1,1,1
2,3.0,106523,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,81K7AmYRz66AZmyO,1,1,1,1,1,0,0,1,1,1
3,4.0,106524,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,RWLV2ZQqQjzo5vN3,1,1,1,1,1,1,1,1,1,1
4,5.0,106526,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,zExm2JmdXQ1o6PyK,1,1,1,1,1,1,1,1,1,1


In [72]:
# Keep only IDs that appear in keys_w2 (pilot participants)
pilot_ids = keys_w2["ID"]

response_w2 = response_w2_wide[response_w2_wide["User ID"].isin(pilot_ids)].copy()

response_w2 = response_w2[
    [
        "condition", "User ID",
        "Baseline", "Day 1", "Day 2", "Day 3", "Day 4", "Day 5", "Weekend 1",
        "Day 6", "Day 7", "Day 8", "Day 9", "Day 10", "Weekend 2"
    ]
].reset_index(drop=True)

response_w2

,condition,User ID,Baseline,Day 1,Day 2,Day 3,Day 4,Day 5,Weekend 1,Day 6,Day 7,Day 8,Day 9,Day 10,Weekend 2
0,3.0,106521,1.0,0,0,1,1,1,1.0,1,0,0,0,0,1.0
1,5.0,106522,1.0,0,0,0,0,0,1.0,1,1,1,1,1,1.0
2,3.0,106523,1.0,1,1,1,1,1,1.0,0,0,1,1,1,1.0
3,4.0,106524,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0
4,5.0,106526,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0
5,5.0,106527,1.0,0,0,0,0,0,0.0,0,0,0,0,0,NaN
6,1.0,106531,1.0,0,1,1,1,1,1.0,1,1,1,1,1,1.0
7,1.0,106532,1.0,0,1,1,1,1,1.0,1,1,1,1,1,1.0
8,2.0,106533,99.0,0,0,0,0,0,99.0,0,0,0,0,0,NaN
9,2.0,106555,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0


In [73]:
wave2_resp_wide = response_w2.merge(keys_w2[["ID", "Norstat ID", "condition"]], left_on="User ID", right_on="ID", how ="left")

In [74]:
wave2_resp_wide

,condition_x,User ID,Baseline,Day 1,Day 2,Day 3,Day 4,Day 5,Weekend 1,Day 6,Day 7,Day 8,Day 9,Day 10,Weekend 2,ID,Norstat ID,condition_y
0,3.0,106521,1.0,0,0,1,1,1,1.0,1,0,0,0,0,1.0,106521.0,LDen9K41b1QAElVw,3.0
1,5.0,106522,1.0,0,0,0,0,0,1.0,1,1,1,1,1,1.0,106522.0,YXrqo4l375NoMZdm,5.0
2,3.0,106523,1.0,1,1,1,1,1,1.0,0,0,1,1,1,1.0,106523.0,81K7AmYRz66AZmyO,3.0
3,4.0,106524,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,106524.0,RWLV2ZQqQjzo5vN3,4.0
4,5.0,106526,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,106526.0,zExm2JmdXQ1o6PyK,5.0
5,5.0,106527,1.0,0,0,0,0,0,0.0,0,0,0,0,0,NaN,106527.0,Elb8AvB1Dv5ApLX5,5.0
6,1.0,106531,1.0,0,1,1,1,1,1.0,1,1,1,1,1,1.0,106531.0,QGYe2jXzkV5AgrXp,1.0
7,1.0,106532,1.0,0,1,1,1,1,1.0,1,1,1,1,1,1.0,106532.0,qRZK2gj6ebyobdka,1.0
8,2.0,106533,99.0,0,0,0,0,0,99.0,0,0,0,0,0,NaN,106533.0,nKY7owOmG8noG3v4,2.0
9,2.0,106555,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,106555.0,nKY7oww4YNdoG3v4,2.0


In [45]:
#wave2_resp_wide = wave2_resp_wide.drop(columns=["condition_y"])

#wave2_resp_wide = wave2_resp_wide.drop(columns=["ID"])

#wave2_resp_wide = wave2_resp_wide.drop(42)

#wave2_resp_wide.to_csv(DATA_DIR / "processed" / "wave2" / wave 2 - task completion.csv")

## Importing Mistral TST data

In [80]:
#Importing the mistral data
mistral_w2 = pd.read_csv(MESSAGES_DIR / "wave2" / "0_mistral-large-latest_0.7_all_shots_parsed.csv")

In [81]:
# First look into the data

mistral_w2.head()

,prompt_id,model,temperature,shots,user_id,message_id,neutral_message,tst_message,tst_explanation,tst_id,llm_tst
0,0,mistral-large-latest,0.7,all,11478,0,"Hello! I’m Sasha, nice to meet you! Over the n...","hi i’m sasha, nice to meet you ! over the next...",The conversational style of person X includes ...,1147820241106092258,"{\n ""rewritten_sentence"": ""hi i’m sasha, nice..."
1,0,mistral-large-latest,0.7,all,11478,1,Hey! Thanks for following along for the first ...,hi ! thanks for following along for the first ...,Person X uses a more informal tone with lowerc...,1147820241106092320,"{\n ""rewritten_sentence"": ""hi ! thanks for fo..."
2,0,mistral-large-latest,0.7,all,11478,2,Reducing how much meat we eat can help save a ...,hi reducing how much meat we eat can help save...,"I used 'hi' to start the conversation, as pers...",1147820241106092330,"{\n ""rewritten_sentence"": ""hi reducing how mu..."
3,0,mistral-large-latest,0.7,all,11478,3,"If everyone ate less animal foods, the CO2 fro...","hi if everyone eat less animal foods, the CO2 ...",I maintained the informal tone and used lowerc...,1147820241106092340,"{\n ""rewritten_sentence"": ""hi if everyone eat..."
4,0,mistral-large-latest,0.7,all,11478,4,Animal farming is responsible for 80% of the l...,hi animal farming responsible for 80% of the a...,Person X uses a more casual tone with lowercas...,1147820241106092350,"{\n ""rewritten_sentence"": ""hi animal farming ..."


In [82]:
# Let's have a look at all the messages
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

mistral_w2["tst_message"].head()

0                                                                                 hi i’m sasha, nice to meet you ! over the next two weeks, i will explore healthy and sustainable eating with you. a year ago, i started eating more plant-based foods and learned a lot along the way. it’s not about being perfect; i don’t think we all need to go completely vegan. however, being a bit mindful of how much animal food we eat and drink can be good for our health and the planet. stay tuned, as i will share lots of interesting science facts and give you tips for small changes that can make a big difference.
1                                                                                        hi ! thanks for following along for the first week\n\ndo you know how more people are choosing plant-based eating ? some of my favourite celebrities eat vegan like lewis hamilton, billie eilish, and ariana grande . others like harry styles and lionel messi avoid animal products for the most part\n\ndo you 

In [83]:
# Adjusting some columns
mistral_w2 = mistral_w2.rename(columns={"user_id": "Movez ID"})

In [85]:
# Let's merge messages with the condition each person is in
mistral_w2 = mistral_w2.merge(keys_w2, on= "Movez ID", how="left")

In [87]:
# Replacements needed per condition
replacement_rules = [
    # condition, message_id_range_start, message_id_range_end
    (5, 1, 6),
    (4, 7, 12),
    (2, 1, 12)
]

for cond, start, end in replacement_rules:
    mask = (mistral_w2["condition"] == cond) & (mistral_w2["message_id"].between(start, end))
    mistral_w2.loc[mask, "tst_message"] = mistral_w2.loc[mask, "neutral_message"]


KeyError: 'condition'

In [88]:
mistral_w2[mistral_w2['condition'] == 1].head()

KeyError: 'condition'

In [711]:
# For myself (researchers) I copy messages from user 64938
#jonas_columns = mistral_pilot[mistral_pilot["Movez ID"] == 26817]
#jonas_columns["Movez ID"] = jonas_columns["Movez ID"].replace(26817,11111)
#mistral_w2 = mistral_w2.append(jonas_columns, ignore_index=True)

C:\Users\Jonas\AppData\Local\Temp\ipykernel_3900\1506340914.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jonas_columns["Movez ID"] = jonas_columns["Movez ID"].replace(26817,11111)
C:\Users\Jonas\AppData\Local\Temp\ipykernel_3900\1506340914.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mistral_w2 = mistral_w2.append(jonas_columns, ignore_index=True)


In [89]:
# Tranforming from long format to wide format
mistral_wide = mistral_w2.pivot(index='Movez ID', columns='message_id', values='tst_message')

#Changing the column names of message ID & rename index
mistral_wide = mistral_wide.add_prefix('Q')
mistral_wide = mistral_wide.rename(columns={"message_id": "Participant ID"}, index={'message_id': 'Participant ID'})

#Sort columns according to their new value
q_columns = [col for col in mistral_wide.columns if col.startswith('Q')]
sorted_columns = sorted(q_columns, key=lambda x: int(x[1:]))


In [90]:
# Let's sort the columns according to the day they are shown
mistral_wide = mistral_wide[sorted_columns]

In [91]:
mistral_wide.head()

message_id,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
Movez ID,,,,,,,,,,,,
11478,"hi i’m sasha, nice to meet you ! over the next two weeks, i will explore healthy and sustainable eating with you. a year ago, i started eating more plant-based foods and learned a lot along the way. it’s not about being perfect; i don’t think we all need to go completely vegan. however, being a bit mindful of how much animal food we eat and drink can be good for our health and the planet. stay tuned, as i will share lots of interesting science facts and give you tips for small changes that can make a big difference.","hi fruits and vegetables are filled with important nutrients like vitamins, minerals, and antioxidants that help our bodies fight diseases. cause people who eat mostly plant foods were 39% less likely to get Covid and have fewer hospital visits overall. well every time we buy food ourselves, we get to decide what goes into our bodies. so you could also talk with your friends during lunch and try out some plant-based foods together.","hi reducing how much meat we eat can help save a lot of CO2, which causes climate change. like if we eat meat every other day instead of every day for a year, it saves as much CO2 as driving a car from London to China… and back!\n\nyou know with small changes like this, everyone can help the environment and slow down climate change now without waiting for others! so let’s start small; like we can try to eat meat only once a day, instead of having it at breakfast, lunch, and dinner.","more than 70% of farm animals in the UK are raised on factory farms to keep animal products cheap. on these farms, a chicken has only about the space of a laptop, so there's hardly any room to move. it's like living under a staircase, like Harry Potter, for your whole life. factory farms keep going cause people buy their products. by choosing to eat less meat or plant-based meats when we can, we can send a message to these farms to make better living conditions for their animals.","hi fiber is really important to keep our stomachs healthy, and it’s only in plants, not in animal foods. when we have enough fiber in our meals, we stay full longer and have more energy during the school day, no more tired feeling after lunch.\n\ndo you know changing habits, like what we eat, can be tricky at first. a easy way to start could be to try more plant-based lunches at school to feel more energetic and ready for the afternoon ?","80% of young people believe that living sustainably is very important for their happiness. And 77% say they would choose meat alternatives as part of their sustainable lifestyle.\n\nso good to see how much our generation agrees on this. Even if we're not trying to be completely vegan, each of us can do something to help. For example, we can try the plant-based food options in our school cafeteria or some meat alternatives more often.","hi ! thanks for following along for the first week\n\ndo you know how more people are choosing plant-based eating ? some of my favourite celebrities eat vegan like lewis hamilton, billie eilish, and ariana grande . others like harry styles and lionel messi avoid animal products for the most part\n\ndo you know how new plant-based products and restaurants are appearing everywhere ? it looks like we'll have lots of options in the future\n\nwhy not see if some of your favourite celebrities also eat plant-based ?","hi reducing how much meat, dairy, and sugary foods we eat can really help improve acne (up to 87% less acne in young people). eating lots of vegetables and fruits also keeps our skin hydrated, protects us from getting sunburned, and even stops wrinkles. having clear, glowing skin might be as simple as just eating more veggies and fruits. when do you usually buy food during your day? do you know if there are tasty plant-based options you could try the next time you have a choice?","hi if everyone eat less animal foods, the CO2 from food production go down by 68%. just by changi

In [92]:
# Last we need to replace the Movez ID key with the Avicenna ID key

# Create a dictionary mapping from 'Movez ID' to 'ID'
id_mapping = keys_w2.set_index('Movez ID')['ID'].to_dict()

# Replace the index of mistral_wide using the mapping
mistral_wide.index = mistral_wide.index.map(lambda idx: id_mapping.get(idx, idx))

# We rename the index to "ID"
mistral_wide.index.name = "ID"


In [93]:
# Lets have a final look into the data

mistral_wide.head()

message_id,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
ID,,,,,,,,,,,,
106887.0,"hi i’m sasha, nice to meet you ! over the next two weeks, i will explore healthy and sustainable eating with you. a year ago, i started eating more plant-based foods and learned a lot along the way. it’s not about being perfect; i don’t think we all need to go completely vegan. however, being a bit mindful of how much animal food we eat and drink can be good for our health and the planet. stay tuned, as i will share lots of interesting science facts and give you tips for small changes that can make a big difference.","hi fruits and vegetables are filled with important nutrients like vitamins, minerals, and antioxidants that help our bodies fight diseases. cause people who eat mostly plant foods were 39% less likely to get Covid and have fewer hospital visits overall. well every time we buy food ourselves, we get to decide what goes into our bodies. so you could also talk with your friends during lunch and try out some plant-based foods together.","hi reducing how much meat we eat can help save a lot of CO2, which causes climate change. like if we eat meat every other day instead of every day for a year, it saves as much CO2 as driving a car from London to China… and back!\n\nyou know with small changes like this, everyone can help the environment and slow down climate change now without waiting for others! so let’s start small; like we can try to eat meat only once a day, instead of having it at breakfast, lunch, and dinner.","more than 70% of farm animals in the UK are raised on factory farms to keep animal products cheap. on these farms, a chicken has only about the space of a laptop, so there's hardly any room to move. it's like living under a staircase, like Harry Potter, for your whole life. factory farms keep going cause people buy their products. by choosing to eat less meat or plant-based meats when we can, we can send a message to these farms to make better living conditions for their animals.","hi fiber is really important to keep our stomachs healthy, and it’s only in plants, not in animal foods. when we have enough fiber in our meals, we stay full longer and have more energy during the school day, no more tired feeling after lunch.\n\ndo you know changing habits, like what we eat, can be tricky at first. a easy way to start could be to try more plant-based lunches at school to feel more energetic and ready for the afternoon ?","80% of young people believe that living sustainably is very important for their happiness. And 77% say they would choose meat alternatives as part of their sustainable lifestyle.\n\nso good to see how much our generation agrees on this. Even if we're not trying to be completely vegan, each of us can do something to help. For example, we can try the plant-based food options in our school cafeteria or some meat alternatives more often.","hi ! thanks for following along for the first week\n\ndo you know how more people are choosing plant-based eating ? some of my favourite celebrities eat vegan like lewis hamilton, billie eilish, and ariana grande . others like harry styles and lionel messi avoid animal products for the most part\n\ndo you know how new plant-based products and restaurants are appearing everywhere ? it looks like we'll have lots of options in the future\n\nwhy not see if some of your favourite celebrities also eat plant-based ?","hi reducing how much meat, dairy, and sugary foods we eat can really help improve acne (up to 87% less acne in young people). eating lots of vegetables and fruits also keeps our skin hydrated, protects us from getting sunburned, and even stops wrinkles. having clear, glowing skin might be as simple as just eating more veggies and fruits. when do you usually buy food during your day? do you know if there are tasty plant-based options you could try the next time you have a choice?","hi if everyone eat less animal foods, the CO2 from food production go down by 68%. just by changing 

In [94]:
# Let's check if all the message cells have a text within them

for index, row in mistral_wide.iterrows():
    # Check for empty values in the row
    for column in mistral_wide.columns:
        if pd.isna(row[column]):  # Check if the value is NaN (empty)
            print(f"Empty value found in column '{column}' at row {index}")

Empty value found in column 'Q12' at row 106595.0
Empty value found in column 'Q3' at row 106917.0
Empty value found in column 'Q12' at row 107182.0
Empty value found in column 'Q12' at row 90310.0


In [95]:
mistral_wide.at[106595, "Q12"] = "Animal farming is behind 80% of the Amazon rainforest loss, you know, 'cause farmers need space for cattle and their food. Last year, the Amazon lost as many trees as there are in the UK and Ireland combined. Imagine if there were no trees left here!By eating less meat (like choosing veggie burgers for lunch), we can make a difference without much change. If you eat with others, you can set an example and inspire your friends and classmates to be more mindful about their food choices too."

mistral_wide.at[106917, "Q3"] = "So, reducing how much meat we eat can actually help save a lot of CO2, which causes climate change. For instance, if we eat meat every other day instead of every day for a year, it saves as much CO2 as driving a car from London to China... and back! ðŸ˜Š"

mistral_wide.at[107182, "Q12"] = "Hi, you know animal farming is why the Amazon rainforest is disappearing, right? Like, it's responsible for 80% of the loss! Farmers need space for cattle and their food. Last year, the Amazon lost as many trees as there are in the UK and Ireland together. Imagine if there were no trees left here! But hey, by eating less meat, like choosing veggie burgers for lunch, we can make a difference without much effort. Plus, if you eat with others, you can set an example and inspire your friends and classmates to be more mindful about their food choices too."

mistral_wide.at[90310, "Q12"] = "Yo, animal farming is responsible for 80% of the Amazon rainforest loss, 'cause farmers need space for cattle and their food. Last year, the Amazon lost as many trees as there are in the UK and Ireland combined. Imagine if there were no trees left here! Just saying. By eating less meat, like choosing veggie burgers for lunch, we can make a difference without much change. If you eat with others, you can set an example and inspire your friends and classmates to be more mindful about their food choices too. What do you think?"

In [96]:
mistral_wide.head()

message_id,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
ID,,,,,,,,,,,,
106887.0,"hi i’m sasha, nice to meet you ! over the next two weeks, i will explore healthy and sustainable eating with you. a year ago, i started eating more plant-based foods and learned a lot along the way. it’s not about being perfect; i don’t think we all need to go completely vegan. however, being a bit mindful of how much animal food we eat and drink can be good for our health and the planet. stay tuned, as i will share lots of interesting science facts and give you tips for small changes that can make a big difference.","hi fruits and vegetables are filled with important nutrients like vitamins, minerals, and antioxidants that help our bodies fight diseases. cause people who eat mostly plant foods were 39% less likely to get Covid and have fewer hospital visits overall. well every time we buy food ourselves, we get to decide what goes into our bodies. so you could also talk with your friends during lunch and try out some plant-based foods together.","hi reducing how much meat we eat can help save a lot of CO2, which causes climate change. like if we eat meat every other day instead of every day for a year, it saves as much CO2 as driving a car from London to China… and back!\n\nyou know with small changes like this, everyone can help the environment and slow down climate change now without waiting for others! so let’s start small; like we can try to eat meat only once a day, instead of having it at breakfast, lunch, and dinner.","more than 70% of farm animals in the UK are raised on factory farms to keep animal products cheap. on these farms, a chicken has only about the space of a laptop, so there's hardly any room to move. it's like living under a staircase, like Harry Potter, for your whole life. factory farms keep going cause people buy their products. by choosing to eat less meat or plant-based meats when we can, we can send a message to these farms to make better living conditions for their animals.","hi fiber is really important to keep our stomachs healthy, and it’s only in plants, not in animal foods. when we have enough fiber in our meals, we stay full longer and have more energy during the school day, no more tired feeling after lunch.\n\ndo you know changing habits, like what we eat, can be tricky at first. a easy way to start could be to try more plant-based lunches at school to feel more energetic and ready for the afternoon ?","80% of young people believe that living sustainably is very important for their happiness. And 77% say they would choose meat alternatives as part of their sustainable lifestyle.\n\nso good to see how much our generation agrees on this. Even if we're not trying to be completely vegan, each of us can do something to help. For example, we can try the plant-based food options in our school cafeteria or some meat alternatives more often.","hi ! thanks for following along for the first week\n\ndo you know how more people are choosing plant-based eating ? some of my favourite celebrities eat vegan like lewis hamilton, billie eilish, and ariana grande . others like harry styles and lionel messi avoid animal products for the most part\n\ndo you know how new plant-based products and restaurants are appearing everywhere ? it looks like we'll have lots of options in the future\n\nwhy not see if some of your favourite celebrities also eat plant-based ?","hi reducing how much meat, dairy, and sugary foods we eat can really help improve acne (up to 87% less acne in young people). eating lots of vegetables and fruits also keeps our skin hydrated, protects us from getting sunburned, and even stops wrinkles. having clear, glowing skin might be as simple as just eating more veggies and fruits. when do you usually buy food during your day? do you know if there are tasty plant-based options you could try the next time you have a choice?","hi if everyone eat less animal foods, the CO2 from food production go down by 68%. just by changing 

In [97]:
# Assuming the index is a string type
invalid_indexes = mistral_wide.index[(mistral_wide.index < 10000) | (mistral_wide.index >= 20000)]

print("Indexes that are not five digits or do not start with '1':")
print(invalid_indexes)

Indexes that are not five digits or do not start with '1':
Index([106887.0, 107223.0, 106521.0, 106527.0, 106905.0, 106594.0, 107186.0,
        28754.0, 106916.0, 106927.0, 106921.0, 106595.0, 107285.0, 106917.0,
       106555.0, 106524.0, 106523.0, 107179.0, 106649.0, 107002.0, 107178.0,
       106531.0, 106532.0, 107183.0, 106629.0, 107053.0, 106522.0, 106526.0,
       106632.0, 107180.0, 107065.0,  74998.0, 107228.0,  77803.0, 107182.0,
       106574.0, 106601.0,  90310.0, 106596.0, 106602.0, 106958.0, 106600.0,
       106886.0, 107185.0],
      dtype='float64', name='ID')


In [98]:
mistral_wide = mistral_wide.drop(index=[28754, 74998, 77803, 90310])

In [761]:
#mistral_wide.to_csv(MESSAGES_DIR / "wave2" / messages_w2.csv', encoding='utf-8', index=True)

In [768]:
#keys_w22.to_csv("keys_w2.csv")